In [1]:
import gym
import ptan

import ptan.ignite as ptan_ignite
from datetime import datetime, timedelta
import random
import warnings

import torch
import torch.optim as optim

from ignite.engine import Engine
from ignite.metrics import RunningAverage
from ignite.contrib.handlers import tensorboard_logger as tb_logger

from lib import dqn_model, common


NAME = "02_n_envs"

In [2]:
def batch_generator(buffer: ptan.experience.ExperienceReplayBuffer,
                    initial: int, batch_size: int, steps: int):
    buffer.populate(initial)
    while True:
        buffer.populate(steps)
        yield buffer.sample(batch_size)

In [6]:
#getting rid of missing metrics warning
warnings.simplefilter("ignore", category=UserWarning)

random.seed(common.SEED)
torch.manual_seed(common.SEED)
params = common.HYPERPARAMS['pong']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

envs = []
n_envs = 3
for _ in range(n_envs):
    env = gym.make(params.env_name)
    env = ptan.common.wrappers.wrap_dqn(env)
    env.seed(common.SEED)
    envs.append(env)

In [7]:
params.batch_size *= n_envs
net = dqn_model.DQN(env.observation_space.shape, env.action_space.n).to(device)

tgt_net = ptan.agent.TargetNet(net)
selector = ptan.actions.EpsilonGreedyActionSelector(epsilon=params.epsilon_start)
epsilon_tracker = common.EpsilonTracker(selector, params)
agent = ptan.agent.DQNAgent(net, selector, device=device)

exp_source = ptan.experience.ExperienceSourceFirstLast(
    env, agent, gamma=params.gamma)
buffer = ptan.experience.ExperienceReplayBuffer(
    exp_source, buffer_size=params.replay_size)
optimizer = optim.Adam(net.parameters(), lr = params.learning_rate)

In [8]:
def process_batch(engine, batch):
    optimizer.zero_grad()
    loss_v = common.calc_loss_dqn(batch, net, tgt_net.target_model,
                                  gamma=params.gamma, device=device)
    loss_v.backward()
    optimizer.step()
    epsilon_tracker.frame(engine.state.iteration)
    if engine.state.iteration % params.target_net_sync == 0:
        tgt_net.sync()
    return {
        "loss": loss_v.item(),
        "epsilon": selector.epsilon,
    }

engine=Engine(process_batch)
ptan_ignite.EndOfEpisodeHandler(exp_source, bound_avg_reward=15.0).attach(engine)
ptan_ignite.EpisodeFPSHandler(fps_mul=n_envs).attach(engine)

In [9]:
@engine.on(ptan_ignite.EpisodeEvents.EPISODE_COMPLETED)
def episode_completed(trainer: Engine):
    print("Episode %d: reward=%s, steps=%s, speed=%.3f frames/s, elapsed=%s" % (
        trainer.state.episode, trainer.state.episode_reward,
        trainer.state.episode_steps, trainer.state.metrics.get('avg_fps', 0),
        timedelta(seconds=trainer.state.metrics.get('time_passed', 0))))
    
@engine.on(ptan_ignite.EpisodeEvents.BOUND_REWARD_REACHED)
def game_solved(trainer: Engine):
    print("Game solved in %s, after %d episodes and %d iterations!" % (
        timedelta(seconds=trainer.state.metrics['time_passed']),
        trainer.state.episode, trainer.state.iteration))
    trainer.should_terminate = True
    
logdir = f"runs/{datetime.now().minute}-{params.run_name}-{NAME}"
tb = tb_logger.TensorboardLogger(log_dir=logdir)
RunningAverage(output_transform=lambda v: v['loss']).attach(engine, "avg_loss")

episode_handler = tb_logger.OutputHandler(tag='episodes', metric_names=['reward', 'steps', 'avg_reward'])
tb.attach(engine, log_handler=episode_handler, event_name=ptan_ignite.EpisodeEvents.EPISODE_COMPLETED)

# writing to tensorboard every 100 iterations
ptan_ignite.PeriodicEvents().attach(engine)
handler = tb_logger.OutputHandler(tag="train", metric_names=['avg_loss', 'avg_fps'],
                                  output_transform=lambda a: a)
tb.attach(engine, log_handler=handler, event_name=ptan_ignite.PeriodEvents.ITERS_100_COMPLETED)

engine.run(batch_generator(buffer, params.replay_initial, params.batch_size, n_envs))

Episode 1: reward=-21.0, steps=848, speed=0.000 frames/s, elapsed=0:02:19.094739
Episode 2: reward=-20.0, steps=1048, speed=0.000 frames/s, elapsed=0:02:19.095739
Episode 3: reward=-21.0, steps=788, speed=0.000 frames/s, elapsed=0:02:19.095739
Episode 4: reward=-19.0, steps=992, speed=0.000 frames/s, elapsed=0:02:19.095739
Episode 5: reward=-21.0, steps=834, speed=0.000 frames/s, elapsed=0:02:19.096739
Episode 6: reward=-20.0, steps=922, speed=0.000 frames/s, elapsed=0:02:19.096739
Episode 7: reward=-21.0, steps=994, speed=0.000 frames/s, elapsed=0:02:19.096739
Episode 8: reward=-21.0, steps=940, speed=0.000 frames/s, elapsed=0:02:19.097782
Episode 9: reward=-20.0, steps=864, speed=0.000 frames/s, elapsed=0:02:19.097782
Episode 10: reward=-21.0, steps=937, speed=0.000 frames/s, elapsed=0:02:19.097782
Episode 11: reward=-21.0, steps=879, speed=64.433 frames/s, elapsed=0:02:19.796381
Episode 12: reward=-20.0, steps=973, speed=65.010 frames/s, elapsed=0:02:30.212696
Episode 13: reward=-21

State:
	iteration: 215170
	epoch: 1
	epoch_length: <class 'NoneType'>
	max_epochs: 1
	output: <class 'dict'>
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'generator'>
	seed: <class 'NoneType'>
	times: <class 'dict'>
	episode: 362
	episode_reward: 19.0
	episode_steps: 1966